# Chapter 7 RNN을 사용한 문장 생성

1. 말뭉치
2. 개선된 언어 모델
3. seq2seq

## 7.1 언어 모델을 사용한 문장 생성

### 7.1.1 RNN을 사용한 문장 생성의 순서

언어 모델 - 단어의 확률 분포 출력  

#### 단어의 확률 분포를 이용해서 다음 단어를 새로 생성하는 방법
1. 결정적 방법  
확률이 가장 높은 단어를 출력, 매번 동일한 단어를 생성

2. 확률적 방법  
단어의 확률 분포를 기반으로 단어를 샘플링, 매번 선택되는 단어가 달라짐

생성된 단어를 언어 모델에 입력해서 원하는 만큼 반복하면서 단어를 계속 생성한다.

### 7.1.2 문장 생성 구현

In [1]:
# Rnnlm 클래스를 상속해 RnnlmGen 클래스를 만든다.
import numpy as np
from common.functions import softmax
from rnnlm import Rnnlm
from better_rnnlm import BetterRnnlm

In [4]:
class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        ''' 언어 모델을 통해 문장을 생성하는 메소드
        
        Parameters
        ----------
        start_id: 최초로 주어지는 단어 ID
        skip_id: 샘플링되지 않게 할 단어 리스트(PTB 데이터셋의 <unk>나 N 등 전처리된 단어를 샘플링하지 않도록)
        sample_size: 샘플링하는 단어 수 
        
        Return
        ----------
        word_ids: 생성된 단어 ID 리스트
        '''
        
        word_ids = [start_id]
        
        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)    # 미니배치로 처리하기 때문에 2차원 배열이어야 함
            score = self.predict(x)
            p = softmax(score.flatten())
            
            sampled = np.random.choice(len(p), size=1, p=p)
            if(skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))
                
        return word_ids

In [6]:
# 학습 없이 단어를 생성
from rnnlm_gen import RnnlmGen
from dataset import ptb